In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,3527
2,Huelva,Confirmados PDIA 14 días,1474
3,Huelva,Tasa PDIA 14 días,"288,5991584808798"
4,Huelva,Confirmados PDIA 7 días,895
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,27
629,Municipio de Huelva sin especificar,Total Confirmados,58
630,Municipio de Huelva sin especificar,Curados,9


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  3527.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  936.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 563 personas en los últimos 7 días 

Un positivo PCR cada 372 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,3527.0,895.0,1474.0,510743.0,175.234903,288.599158,106.0
Huelva-Costa,1781.0,441.0,719.0,288115.0,153.063881,249.553130,64.0
Condado-Campiña,1113.0,261.0,416.0,155057.0,168.325197,268.288436,58.0
Huelva (capital),936.0,255.0,386.0,143663.0,177.498730,268.684352,33.0
Sierra de Huelva-Andévalo Central,575.0,166.0,312.0,67571.0,245.667520,461.736544,21.0
Gibraleón,100.0,45.0,61.0,12607.0,356.944555,483.858174,15.0
Aljaraque,136.0,35.0,52.0,21260.0,164.628410,244.590781,13.0
Almonte,170.0,64.0,103.0,24191.0,264.561200,425.778182,13.0
Valverde del Camino,102.0,60.0,69.0,12820.0,468.018721,538.221529,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),33.0,2.0,21.0,516.0,387.596899,4069.767442,1.0
Cortegana,111.0,29.0,96.0,4666.0,621.517360,2057.436777,2.0
Escacena del Campo,42.0,22.0,24.0,2284.0,963.222417,1050.788091,1.0
Villarrasa,49.0,15.0,22.0,2176.0,689.338235,1011.029412,4.0
Almonaster la Real,19.0,7.0,17.0,1819.0,384.826828,934.579439,1.0
Cerro de Andévalo (El),25.0,3.0,18.0,2364.0,126.903553,761.421320,0.0
Calañas,23.0,19.0,20.0,2791.0,680.759584,716.589036,1.0
Palma del Condado (La),164.0,36.0,63.0,10761.0,334.541399,585.447449,5.0
Valverde del Camino,102.0,60.0,69.0,12820.0,468.018721,538.221529,11.0
